In [ ]:
import pandas as pd
import os
from pathlib import Path

# Đường dẫn gốc
BASE_PATH = Path("H:/SentimentAnalystSchool/MidtermExam/DatasetUnder80kWord")

# Định nghĩa các file cần gộp
FILES_CONFIG = {
    "Amazon": {
        "train": BASE_PATH / "Amazon" / "train_reviews_under80words.csv",
        "dev": BASE_PATH / "Amazon" / "dev_reviews_under80words.csv",
        "test": BASE_PATH / "Amazon" / "test_reviews_under80words.csv"
    },
    "Hotels": {
        "train": BASE_PATH / "Hotels" / "train_reviews_under80words.csv",
        "dev": BASE_PATH / "Hotels" / "dev_reviews_under80words.csv",
        "test": BASE_PATH / "Hotels" / "test_reviews_under80words.csv"
    },
    "AmazonsElectronics": {
        "all": BASE_PATH / "AmazonsElectronics" / "output_with_aspects.csv"
    }
}

print("Cấu hình đường dẫn files:")
for source, files in FILES_CONFIG.items():
    print(f"\n{source}:")
    for file_type, path in files.items():
        exists = "✓" if path.exists() else "✗"
        print(f"  {file_type}: {path.name} [{exists}]")

In [ ]:
def load_and_standardize_data(file_path, source_name, file_type="train"):
    """
    Đọc file CSV và chuẩn hóa cột
    
    Parameters:
    - file_path: đường dẫn file
    - source_name: tên nguồn dữ liệu (Amazon, Hotels, etc.)
    - file_type: loại file (train, dev, test, all)
    
    Returns:
    - DataFrame đã chuẩn hóa
    """
    try:
        df = pd.read_csv(file_path)
        
        # Chuẩn hóa tên cột
        df.columns = df.columns.str.strip().str.lower()
        
        # Map các tên cột có thể khác nhau
        column_mapping = {
            'reviewtext': 'text',
            'review': 'text',
            'aspectterm': 'aspect',
            'aspect': 'aspect',
            'sentiment': 'sentiment'
        }
        
        df = df.rename(columns=column_mapping)
        
        # Thêm cột metadata
        df['source'] = source_name
        df['split'] = file_type
        
        # Chỉ giữ các cột cần thiết
        required_cols = ['text', 'aspect', 'sentiment', 'source', 'split']
        available_cols = [col for col in required_cols if col in df.columns]
        df = df[available_cols]
        
        print(f"  ✓ Loaded {len(df)} rows from {file_path.name}")
        return df
        
    except Exception as e:
        print(f"  ✗ Error loading {file_path}: {e}")
        return pd.DataFrame()

print("Hàm load_and_standardize_data đã được định nghĩa!")

In [ ]:
def merge_all_datasets(files_config, output_path=None):
    """
    Gộp tất cả các dataset thành một file duy nhất
    
    Parameters:
    - files_config: dict chứa cấu hình các file
    - output_path: đường dẫn lưu file output (optional)
    
    Returns:
    - DataFrame đã gộp
    """
    all_dataframes = []
    
    print("=" * 60)
    print("BẮT ĐẦU GỘP DATASETS")
    print("=" * 60)
    
    for source_name, files in files_config.items():
        print(f"\n📂 Loading from {source_name}:")
        for file_type, file_path in files.items():
            if file_path.exists():
                df = load_and_standardize_data(file_path, source_name, file_type)
                if not df.empty:
                    all_dataframes.append(df)
            else:
                print(f"  ⚠ File not found: {file_path}")
    
    if not all_dataframes:
        print("\n❌ Không có dữ liệu nào được load!")
        return pd.DataFrame()
    
    # Gộp tất cả
    merged_df = pd.concat(all_dataframes, ignore_index=True)
    
    print("\n" + "=" * 60)
    print("KẾT QUẢ GỘP DATASET")
    print("=" * 60)
    print(f"\n📊 Tổng số mẫu: {len(merged_df):,}")
    print(f"\n📈 Phân bố theo nguồn:")
    print(merged_df['source'].value_counts().to_string())
    print(f"\n📈 Phân bố theo split:")
    print(merged_df['split'].value_counts().to_string())
    print(f"\n📈 Phân bố theo sentiment:")
    print(merged_df['sentiment'].value_counts().to_string())
    
    # Lưu file nếu có output_path
    if output_path:
        merged_df.to_csv(output_path, index=False)
        print(f"\n💾 Đã lưu file: {output_path}")
    
    return merged_df

print("Hàm merge_all_datasets đã được định nghĩa!")

In [ ]:
# Thực hiện gộp dataset
OUTPUT_PATH = BASE_PATH / "merged_dataset_all.csv"

merged_df = merge_all_datasets(FILES_CONFIG, output_path=OUTPUT_PATH)

In [ ]:
# Xem một số mẫu dữ liệu
print("📋 Một số mẫu dữ liệu đầu tiên:")
merged_df.head(10)

In [ ]:
def split_merged_dataset(merged_df, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, 
                         output_dir=None, random_state=42):
    """
    Chia dataset đã gộp thành train/val/test
    
    Parameters:
    - merged_df: DataFrame đã gộp
    - train_ratio, val_ratio, test_ratio: tỉ lệ chia
    - output_dir: thư mục lưu file (optional)
    - random_state: seed cho random
    
    Returns:
    - train_df, val_df, test_df
    """
    from sklearn.model_selection import train_test_split
    
    # Shuffle data
    df = merged_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    # Chia train và temp (val + test)
    train_df, temp_df = train_test_split(
        df, 
        train_size=train_ratio,
        random_state=random_state,
        stratify=df['sentiment'] if 'sentiment' in df.columns else None
    )
    
    # Chia temp thành val và test
    val_size = val_ratio / (val_ratio + test_ratio)
    val_df, test_df = train_test_split(
        temp_df,
        train_size=val_size,
        random_state=random_state,
        stratify=temp_df['sentiment'] if 'sentiment' in temp_df.columns else None
    )
    
    print(f"\n📊 Kết quả chia dataset:")
    print(f"  Train: {len(train_df):,} mẫu ({train_ratio*100:.0f}%)")
    print(f"  Val:   {len(val_df):,} mẫu ({val_ratio*100:.0f}%)")
    print(f"  Test:  {len(test_df):,} mẫu ({test_ratio*100:.0f}%)")
    
    # Lưu file nếu có output_dir
    if output_dir:
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
        
        train_df.to_csv(output_dir / "train_merged.csv", index=False)
        val_df.to_csv(output_dir / "val_merged.csv", index=False)
        test_df.to_csv(output_dir / "test_merged.csv", index=False)
        
        print(f"\n💾 Đã lưu files vào: {output_dir}")
    
    return train_df, val_df, test_df

print("Hàm split_merged_dataset đã được định nghĩa!")

In [ ]:
# Chia dataset thành train/val/test với tỉ lệ 80/10/10
OUTPUT_DIR = BASE_PATH / "merged"

train_df, val_df, test_df = split_merged_dataset(
    merged_df, 
    train_ratio=0.8, 
    val_ratio=0.1, 
    test_ratio=0.1,
    output_dir=OUTPUT_DIR
)

In [ ]:
# Kiểm tra phân bố sentiment trong từng split
print("📊 Phân bố sentiment trong từng split:\n")

for name, df in [("Train", train_df), ("Val", val_df), ("Test", test_df)]:
    print(f"{name}:")
    print(df['sentiment'].value_counts(normalize=True).apply(lambda x: f"{x:.2%}"))
    print()

## Tổng kết

Các file đã được tạo:
- `merged_dataset_all.csv`: File gộp tất cả dữ liệu
- `merged/train_merged.csv`: Tập train (80%)
- `merged/val_merged.csv`: Tập validation (10%)  
- `merged/test_merged.csv`: Tập test (10%)

Cấu trúc cột của dataset đã chuẩn hóa:
- `text`: Nội dung review
- `aspect`: Aspect term được trích xuất
- `sentiment`: Nhãn sentiment (positive/negative/neutral)
- `source`: Nguồn dữ liệu (Amazon/Hotels/AmazonsElectronics)
- `split`: Loại split gốc (train/dev/test/all)